In [1]:
import sys
import pandas as pd 
sys.path.insert(0, '/home/bekah/gPhoton2')
from gPhoton.pipeline import execute_pipeline
from astropy.io import fits
from io import BytesIO
import numpy as np 
import os
import matplotlib.pyplot as plt
from gPhoton.coadd import zero_flag_and_edge
import pyarrow.parquet as parquet
from quickbin import bin2d


In [2]:
execute_pipeline(
                14112 ,#int(eclipse),
                "FUV",
                depth=None,
                # integer; None to deactivate (default None)
                threads=4,
                # where to both write output data and look for input data
                local_root="/home/bekah/gPhoton2/test_data",
                # auxiliary remote location for input data
                # remote_root="/mnt/s3",
                recreate=True,
                # list of floats; relevant only to lightcurve / photometry portion
                aperture_sizes=[12.8],
                # actually write image/movie products? otherwise hold in memory but
                # discard (possibly after performing photometry).
                write={"movie": False, "image": False},
                coregister_lightcurves=False,
                # photonpipe, moviemaker, None (default None)
                stop_after=None,
                photometry_only=False,
                # None, "gzip", "rice"
                compression="rice",
                # use array sparsification on movie frames?
                lil=False,
                # write movie frames as separate files
                burst=False,
                extended_photonlist=True,
                # aspect file, don't need to set unless need to use alt
                # file, 'aspect2.parquet'
            )
        

/home/bekah/gPhoton2/gPhoton/pipeline.py:231: UserWarning: lil=False is deprected and will be removed in a future release. Defaulting to lil=True.
  warnings.warn("lil=False is deprected and will be removed in a future release. Defaulting to lil=True.")


starting timer
eclipse 14112 FUV  -- GII; 0 leg(s)
downloading raw6file
Processing eclipse 14112
Offsetting FUV image for eclipse 14112 at 29.7843246809265 degrees.
Setting FUV offsets to x=-0.02999, y=0.04301841227073827
trange= ( 819078522.995 , 819079247.995 )                   
RA AVG: 66.95414560550891, DEC AVG: 17.084464825118285, ROLL AVG: 236.99848085491396
Loading raw6 file...                                        
5807190 events
Unpacking raw6 data...                                      
stim_coef0, stim_coef1 = -237970.18854583672, 0.00029674785653263186
[0.50578046 0.50578046 0.50578046 ... 0.50578042 0.50578042 0.50578042]
[0.50578057 0.50578057 0.50578057 ... 0.50578053 0.50578053 0.50578053]
[0.50578053 0.50578053 0.50578053 ... 0.5057805  0.5057805  0.5057805 ]
[0.5057805  0.5057805  0.5057805  ... 0.50578046 0.50578046 0.50578046]
[0.50578039 0.50578039 0.50578039 ... 0.50578036 0.50578036 0.50578036]
[0.50578042 0.50578042 0.50578042 ... 0.50578039 0.50578039 0.5057

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
eclipses = pd.read_csv("/home/bekah/glcat/masks/every3_fuv.csv")

In [ ]:
eclipses = eclipses[1000:1200]

In [ ]:
eclipses

In [ ]:
for index, eclipse in eclipses['eclipse'].items():
    print(eclipse)
    try: 
        execute_pipeline(
                14112 ,#int(eclipse),
                "FUV",
                depth=None,
                # integer; None to deactivate (default None)
                threads=4,
                # where to both write output data and look for input data
                local_root="/home/bekah/gPhoton2/test_data",
                # auxiliary remote location for input data
                # remote_root="/mnt/s3",
                recreate=True,
                # list of floats; relevant only to lightcurve / photometry portion
                aperture_sizes=[12.8],
                # actually write image/movie products? otherwise hold in memory but
                # discard (possibly after performing photometry).
                write={"movie": False, "image": False},
                coregister_lightcurves=False,
                # photonpipe, moviemaker, None (default None)
                stop_after=None,
                photometry_only=False,
                # None, "gzip", "rice"
                compression="rice",
                # use array sparsification on movie frames?
                lil=False,
                # write movie frames as separate files
                burst=False,
                extended_photonlist=True,
                # aspect file, don't need to set unless need to use alt
                # file, 'aspect2.parquet'
            )
        
         # photon table would already exist in gphoton2 
        photons = parquet.read_table(f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-nd-b00.parquet", 
                                     columns=['ra','dec','col','row','q','flags','detrad','t']).to_pandas()

        # want photons that are on detector, no flags, has aspect soln 
        photons = photons[(photons['detrad'] < 600)]
        photons = photons.dropna(subset=['ra', 'dec'])
        photons = photons[(photons['flags'] < 1)]

        # bin2d for ra & dec std + count, also get vals for q range 
        ra = bin2d(photons['col'],photons['row'],photons['ra'],n_bins=800,op=('count', 'std'))
        dec = bin2d(photons['col'],photons['row'],photons['dec'],n_bins=800,op=('std'))
        q_m =  bin2d(photons['col'],photons['row'],photons['q'],n_bins=800,op=('min', 'max'))

        # dispersion metric 
        disp = ra['std']+dec # bin2d only returns a dictionary when more than one stat is calculated 

        # rough exp time (use the correct one for threshold calc)
        time = abs(photons['t'].iloc[0]-photons['t'].iloc[-1])

        fig, axs = plt.subplots(2, 2, figsize=(10, 8))

        axs[0,0].scatter(disp,ra['count']/time,s=.1)
        axs[0,0].set_ylabel("total photons per bin div by expt")
        axs[0,0].set_xlabel('dispersion per bin')

        axs[1,0].hist(np.ravel(disp))
        axs[1,0].set_xlabel("dispersion")

        axs[0,1].imshow(disp)
        axs[0,1].set_xlabel("dispersion")

        axs[1,1].imshow(q_m['max']-q_m['min'])
        axs[1,1].set_xlabel('q range')

        plt.tight_layout()
        plt.savefig(f'/home/bekah/gPhoton2/test_data/e{eclipse}_fuv.png')
    except Exception as e: 
        print(e)


In [ ]:
# so I want to show the current (background rms) method for threshold as well as other ways 
# other ways include: 
# using dispersion as the metric
# but how do I turn that into a map? 

In [ ]:
from astropy.io import fits
from io import BytesIO
import numpy as np 
import os
import matplotlib.pyplot as plt
from gPhoton.coadd import zero_flag_and_edge
import pyarrow.parquet as parquet

In [ ]:
from quickbin import bin2d
import pyarrow.parquet as parquet
import pandas as pd 

# photon table would already exist in gphoton2 
photons = parquet.read_table("/media/bekah/BekahA/background_test/e00580/e00580-nd-b00.parquet", 
                             columns=['ra','dec','col','row','q','flags','detrad','t']).to_pandas()

# want photons that are on detector, no flags, has aspect soln 
photons = photons[(photons['detrad'] < 600)]
photons = photons.dropna(subset=['ra', 'dec'])
photons = photons[(photons['flags'] < 1)]

# bin2d for ra & dec std + count, also get vals for q range 
ra = bin2d(photons['col'],photons['row'],photons['ra'],n_bins=800,op=('count', 'std'))
dec = bin2d(photons['col'],photons['row'],photons['dec'],n_bins=800,op=('std'))
q_m =  bin2d(photons['col'],photons['row'],photons['q'],n_bins=800,op=('min', 'max'))

# dispersion metric 
disp = ra['std']+dec # bin2d only returns a dictionary when more than one stat is calculated 

# rough exp time (use the correct one for threshold calc)
time = abs(photons['t'].iloc[0]-photons['t'].iloc[-1])


In [ ]:
ra

In [ ]:
# getting threshold 
high_disp = np.argwhere(disp>.013)
values = ra['count'][high_disp[:, 0], high_disp[:, 1]]
max(values/time)*.5

In [ ]:
# getting threshold 
high_disp = np.argwhere(disp<.0025)
values = ra['count'][high_disp[:, 0], high_disp[:, 1]]
max(values/time)*.5

In [ ]:
# fig 1: DISPERSION 
fig, axs = plt.subplots(2, 2, figsize=(10, 8))

axs[0,0].scatter(disp,ra['count']/time,s=.1)
axs[0,0].set_ylabel("total photons per bin div by expt")
axs[0,0].set_xlabel('dispersion per bin')

axs[1,0].hist(disp)
axs[1,0].set_xlabel("dispersion")

axs[0,1].imshow(disp)
axs[0,1].set_xlabel("dispersion")

axs[1,1].imshow(q_m['max']-q_m['min'])
axs[1,1].set_xlabel('q range')

plt.tight_layout()
plt.savefig(f'/media/bekah/BekahA/background_test/{eclipse}_nuv.png')

In [ ]:
plt.hist(disp)
plt.xlabel("dispersion")

In [ ]:
800*800

In [ ]:
# getting eclipse list
import re
main_folder = '/media/bekah/BekahA/background_test/pics_and_photonlists'
folders = [
    f for f in os.listdir(main_folder) 
    if os.path.isdir(os.path.join(main_folder, f)) and re.search(r'\d', f)]
eclipses = [folder.replace('e', '') for folder in folders]

In [ ]:
len(eclipses)

In [ ]:
# putting it all together 
from quickbin import bin2d
import pyarrow.parquet as parquet
import pandas as pd 

for eclipse in eclipses: 
    # photon table would already exist in gphoton2 
    photons = parquet.read_table(f"/media/bekah/BekahA/background_test/pics_and_photonlists/e{eclipse}/e{eclipse}-nd-b00.parquet", 
                                 columns=['ra','dec','col','row','q','flags','detrad','t']).to_pandas()

    # want photons that are on detector, no flags, has aspect soln 
    photons = photons[(photons['detrad'] < 600)]
    photons = photons.dropna(subset=['ra', 'dec'])
    photons = photons[(photons['flags'] < 1)]

    # bin2d for ra & dec std + count, also get vals for q range 
    ra = bin2d(photons['col'],photons['row'],photons['ra'],n_bins=800,op=('count', 'std'))
    dec = bin2d(photons['col'],photons['row'],photons['dec'],n_bins=800,op=('std'))
    q_m =  bin2d(photons['col'],photons['row'],photons['q'],n_bins=800,op=('min', 'max'))

    # dispersion metric 
    disp = ra['std']+dec # bin2d only returns a dictionary when more than one stat is calculated 

    # rough exp time (use the correct one for threshold calc)
    time = abs(photons['t'].iloc[0]-photons['t'].iloc[-1])

    fig, axs = plt.subplots(2, 2, figsize=(10, 8))

    axs[0,0].scatter(disp,ra['count']/time,s=.1)
    axs[0,0].set_ylabel("total photons per bin div by expt")
    axs[0,0].set_xlabel('dispersion per bin')

    axs[1,0].hist(np.ravel(disp))
    axs[1,0].set_xlabel("dispersion")

    axs[0,1].imshow(disp)
    axs[0,1].set_xlabel("dispersion")

    axs[1,1].imshow(q_m['max']-q_m['min'])
    axs[1,1].set_xlabel('q range')

    plt.tight_layout()
    plt.savefig(f'/media/bekah/BekahA/background_test/e{eclipse}_fuv.png')


In [ ]:
special_eclipses = ['00587','00826','00767','14182']

from quickbin import bin2d

for eclipse in special_eclipses: 
    # photon table would already exist in gphoton2 
    photons = parquet.read_table(f"/media/bekah/BekahA/background_test/pics_and_photonlists/e{eclipse}/e{eclipse}-nd-b00.parquet", 
                                 columns=['ra','dec','col','row','q','flags','detrad','t']).to_pandas()

    # want photons that are on detector, no flags, has aspect soln 
    photons = photons[(photons['detrad'] < 600)]
    photons = photons.dropna(subset=['ra', 'dec'])
    photons = photons[(photons['flags'] < 1)]

    # bin2d for ra & dec std + count, also get vals for q range 
    ra = bin2d(photons['col'],photons['row'],photons['ra'],n_bins=800,op=('count', 'std'))
    dec = bin2d(photons['col'],photons['row'],photons['dec'],n_bins=800,op=('std'))
    q_m =  bin2d(photons['col'],photons['row'],photons['q'],n_bins=800,op=('min', 'max'))

    # dispersion metric 
    disp = ra['std']+dec # bin2d only returns a dictionary when more than one stat is calculated 

    hdu1 = fits.PrimaryHDU(disp)
    hdu2 = fits.ImageHDU(ra['count'])
    hdulist = fits.HDUList([hdu1, hdu2])
    hdulist.writeto(f'e{eclipse}_hists.fits', overwrite=True)
    print(f"done w {eclipse}")

In [ ]:
from quickbin import bin2d

eclipse = '14112'

# photon table would already exist in gphoton2 
# /media/bekah/BekahA/background_test/pics_and_photonlists
photons = parquet.read_table(f"/home/bekah/gPhoton2/test_data/e{eclipse}/e{eclipse}-fd-b00.parquet", 
                             columns=['ra','dec','col','row','q','flags','detrad','t']).to_pandas()

# want photons that are on detector, no flags, has aspect soln 
photons = photons[(photons['detrad'] < 600)]
photons = photons.dropna(subset=['ra', 'dec'])
photons = photons[(photons['flags'] < 1)]

# bin2d for ra & dec std + count, also get vals for q range 
ra = bin2d(photons['col'],photons['row'],photons['ra'],n_bins=800,op=('count', 'std'))
dec = bin2d(photons['col'],photons['row'],photons['dec'],n_bins=800,op=('std'))
q_m =  bin2d(photons['col'],photons['row'],photons['q'],n_bins=800,op=('min', 'max'))

# dispersion metric 
disp = ra['std']+dec # bin2d only returns a dictionary when more than one stat is calculated 

metric1 = disp / ra['count']
metric2 = ra['count'] / disp
metric3 = ra['count'] * disp

In [ ]:
SEM = disp / np.sqrt(ra['count'])

In [ ]:
plt.hist(np.ravel(SEM),bins=100)

In [ ]:
plt.imshow(SEM > .002)

In [ ]:
photons = photons[(photons['detrad'] < 600)]
photons = photons.dropna(subset=['ra', 'dec'])
photons = photons[(photons['flags'] < 1)]

cnt = bin2d(photons['ra'],photons['dec'],photons['ra'],n_bins=800,op=('count', 'std'))

plt.imshow(centile_clip(cnt['count']),interpolation=None)
plt.colorbar()

In [ ]:
plt.hist(np.ravel(cnt['count']),bins=100)

In [ ]:
plt.imshow(metric3)

In [ ]:
metric2 = np.where(np.isnan(metric2), 0, metric2)
metric2[metric2 > 1e308] = 0

In [ ]:
np.min(metric2)

In [ ]:
type(metric2)

In [ ]:
metric2

In [ ]:
hdu1 = fits.PrimaryHDU()
hdu3 = fits.ImageHDU(SEM)
hdu2 = fits.ImageHDU(metric3)
hdulist = fits.HDUList([hdu1, hdu2, hdu3])
hdulist.writeto(f'e{eclipse}_metrics.fits', overwrite=True)

In [ ]:
# checking threshold map 

e4112 = fits.open("/media/bekah/BekahA/background_test/e14112_FUV_threshold.fits")

In [ ]:
def centile_clip(image, centiles=(1, 99)):
    """
    simple clipping function that clips values above and below a given
    percentile range
    """
    finite = np.ma.masked_invalid(image)
    bounds = np.percentile(finite[~finite.mask].data, centiles)
    result = np.ma.clip(finite, *bounds)
    
    if isinstance(image, np.ma.MaskedArray):
        
        return result
    
    return result.data

In [ ]:
plt.imshow(e4112[1].data>e4112[3].data)

In [ ]:
plt.imshow(centile_clip(e4112[3].data))

In [ ]:
fig, ax = plt.subplots()
plt.imshow(e4112_nosub[1].data>e4112_nosub[3].data)
ax.invert_yaxis()

In [ ]:
plt.imshow(centile_clip(e4112_nosub[1].data))

In [ ]:
from photutils.background import Background2D, MedianBackground
from astropy.stats import SigmaClip


In [ ]:
e4112_nosub = fits.open("/media/bekah/BekahA/background_test/e14112_FUV_threshold_increasedFlat.fits")

In [ ]:
sigma_clip = SigmaClip(sigma=3.)
bkg_estimator = MedianBackground()
bkg = Background2D(cnt_image,
                       (50, 50),
                       filter_size=(3, 3),
                       bkg_estimator=bkg_estimator,
                       sigma_clip=sigma_clip)

In [ ]:
plt.imshow(bkg.background)
plt.colorbar()

In [ ]:
plt.imshow(centile_clip(cnt_image))
plt.colorbar()

In [ ]:
bkg.background.shape

In [ ]:
bkg2.background.shape

In [ ]:
plt.imshow(bkg.background_rms.astype(np.float32))


In [ ]:
plt.imshow(np.multiply(3,bkg.background_rms),interpolation=None,cmap='grey')


In [ ]:
fig, ax = plt.subplots()
plt.imshow(np.multiply(3,bkg2.background_rms),interpolation=None,cmap='grey')
ax.invert_yaxis()

In [ ]:

fig, ax = plt.subplots()
plt.imshow(e4112[1].data>np.multiply(3,bkg2.background_rms))
ax.invert_yaxis()

In [ ]:
type(e4112_nosub[4].data)

In [ ]:
cnt_image = e4112_nosub[1].data
cnt_image[cnt_image == 0] = 0.0015

sigma_clip = SigmaClip(sigma=3.)
bkg_estimator = MedianBackground()
bkg = Background2D(cnt_image,
                       (50, 50),
                       filter_size=(3, 3),
                       bkg_estimator=bkg_estimator,
                       sigma_clip=sigma_clip)

plt.imshow(bkg.background)
plt.colorbar()

In [ ]:
plt.imshow(centile_clip(cnt_image-bkg.background))
plt.colorbar()

In [ ]:
plt.imshow(cnt_image==0)